## 1.Project Introduction

The objective of this case study is to predict the health insurance cost incurred by Individuals based on their age, gender, BMI, number of children, smoking habit and geo-location.

本案例研究的目的是根据年龄，性别，BMI，儿童数量，吸烟习惯和地理位置来预测个人承担的健康保险费用。

- Features available are:

    - sex: insurance contractor gender, female, male

    - bmi: Body mass index (ideally 18.5 to 24.9)

    - children: Number of children covered by health insurance / Number of dependents

    - smoker: smoking habits

    - region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

    - charges: Individual medical costs billed by health insurance


Data Source:https://www.kaggle.com/mirichoi0218/insurance

## 2.Load the Dataset and Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
# convert categorical variable to numerical

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# sex
le.fit(df.sex.drop_duplicates())
df.sex = le.transform(df.sex)

# smoke
le.fit(df.smoker.drop_duplicates())
df.smoker = le.transform(df.smoker)

In [ ]:
df.head()

In [ ]:
# Check unique values in 'region' column
df['region'].unique()

In [ ]:
region_dummies = pd.get_dummies(df['region'], drop_first = True)
region_dummies.head()

In [ ]:
df = pd.concat([df, region_dummies], axis = 1)

In [ ]:
df.drop(['region'], axis = 1, inplace = True)

In [ ]:
df.head()

## 3.Exploratory Data Analysis

In [ ]:
# Check NA values
df.isnull().sum()

In [ ]:
# Check the DataFrame Info
df.info()

The people in the southeast have the highest insurance charges and obviously, with the highest BMI index.

In [ ]:
## group by region

In [ ]:
# Group the Age to see the relationship between age and charges
df_age = df.groupby(by = 'age').mean()
df_age

We can get a basic idea that the charges will increase when the age increase.

plot analysis

In [ ]:
df.describe()

## 4.Visualzie the dataset

In [ ]:
df[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']].hist(bins = 30, figsize = (10,10), color = 'blue')

In [ ]:
#plot pairplot
sns.pairplot(df)

Check the realtionship between the age and Charges, we can see the it is more complex than a linear relationship.

So we will consider some non-linear Algorithim such as ANN or XgBoost.

In [ ]:
# Explore the relation between age and charges
sns.regplot(x = 'age', y = 'charges', data = df)
plt.show()

In [ ]:
# Explore the relation between BMI and charges
sns.regplot(x = 'bmi', y = 'charges', data = df)
plt.show()

In [ ]:
# Check the correlation
corr = df.corr()
corr

In [ ]:
plt.figure(figsize=(8,8))
p = sns.heatmap(df.corr(),cmap="coolwarm",annot=True, square =True)

Smoke have the most positive relationship with the charges

## 5.Split the train test dataset

In [ ]:
df.columns

In [ ]:
X = df.drop(['charges'], axis = 1)
y = df.charges

In [ ]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [ ]:
y_reshape = y.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
X_scale = scaler_x.fit_transform(X)

scaler_y = StandardScaler()
y_scale = scaler_y.fit_transform(y_reshape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scale,y_scale,test_size = 0.2,random_state = 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## 6.Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error,accuracy_score
from math import sqrt

lr = LinearRegression()
lr.fit(X_train, y_train)

lr_R2 = lr.score(X_test,y_test)
lr_R2

In [ ]:
# Predict the value
y_pred = lr.predict(X_test)

y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test)

In [ ]:
k = X_test.shape[1]
n = len(X_test)

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)

print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)

## 7.Multiple linear regression (Degree = 2)

In [ ]:
quad = PolynomialFeatures (degree = 2)
X_quad = quad.fit_transform(X_scale)

X_train_plr,X_test_plr,y_train_plr,y_test_plr = train_test_split(X_quad,y_scale,test_size = 0.2, random_state = 0)

plr = LinearRegression().fit(X_train_plr,y_train_plr)

plr_R2 = plr.score(X_test_plr,y_test_plr)
plr_R2

In [ ]:
# Predict the value
y_pred = plr.predict(X_test_plr)

y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test_plr)

In [ ]:
k = X_test.shape[1]
n = len(X_test)

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)

print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)

## 8.Deep Learning Model (ANN)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [ ]:
ANN_model = keras.Sequential()
ANN_model.add(Dense(50, input_dim = 8))
ANN_model.add(Activation('relu'))

ANN_model.add(Dense(180))
ANN_model.add(Activation('relu'))
ANN_model.add(Dropout(0.3))

ANN_model.add(Dense(180))
ANN_model.add(Activation('relu'))
ANN_model.add(Dropout(0.3))

ANN_model.add(Dense(50))
ANN_model.add(Activation('linear'))

ANN_model.add(Dense(1))
ANN_model.compile(loss = 'mse', optimizer = 'adam')
ANN_model.summary()

In [ ]:
# Compile the model and trainning
ANN_model.compile(optimizer= 'Adam', loss='mean_squared_error')
epochs_hist = ANN_model.fit(X_train, y_train, epochs= 50, batch_size = 20, validation_split= 0.2)

result = ANN_model.evaluate(X_test, y_test)
accuracy_ANN = 1 - result
print('Accuracy : {}'.format(accuracy_ANN) )

In [ ]:
epochs_hist.history.keys()
plt.plot(epochs_hist.history['loss'])
plt.plot(epochs_hist.history['val_loss'])
plt.title('Model Loss Progross During Trainning')
plt.xlabel('Epochs')
plt.ylabel('Training and validation loss')
plt.legend(['Trainnig Loss','Validation Loss'])

In [ ]:
# change the color
y_pred = ANN_model.predict(X_test)

y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test)

plt.plot(y_test_orig, y_pred_orig, "^", color = 'b')
plt.xlabel('Model Predictions')
plt.ylabel('True Values')

In [ ]:
k = X_test.shape[1]
n = len(X_test)

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)

print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)

## 9.Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [4, 5, 6, 7],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [3, 4, 5],
    'n_estimators': [10, 30, 50, 100]
}

# Create a based model
rf = RandomForestRegressor(criterion = 'mse')

# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 4)

In [ ]:
grid_search_rf.fit(X_train, y_train)

grid_search_rf.best_params_

In [ ]:
grid_search_rf_R2 = grid_search_rf.score(X_test,y_test)
grid_search_rf_R2

In [ ]:
# Predict the value
y_pred = grid_search_rf.predict(X_test)

y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test)

In [ ]:
k = X_test.shape[1]
n = len(X_test)

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)

print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)

## 10.XGBoost

In [ ]:
#! pip install xgboost
import xgboost as xgb

In [ ]:
param_grid = {
        'gamma': [0.1,0.3, 0.5, 1],   # regularization parameter 
        'subsample': [0.6, 0.8, 1.0], # % of rows taken to build each tree
        'colsample_bytree': [0.6, 0.8, 1.0], # number of columns used by each tree
        'max_depth': [3, 4, 5], # depth of each tree
        'n_estimaters': [10, 30, 50, 100],
        'learning_rate' :[0.01, 0.03, 0.05]
        }

In [ ]:
xgb_model = xgb.XGBRegressor(objective = 'reg:squarederror')

grid_xgb = GridSearchCV(xgb_model,
                    param_grid,
                    cv = 5, 
                    n_jobs = -1, 
                    verbose = 4)

In [ ]:
grid_xgb.fit(X_train, y_train)

grid_xgb.best_params_

In [ ]:
grid_xgb_R2 = grid_xgb.score(X_test,y_test)
grid_xgb_R2

In [ ]:
# Predict the value
y_pred = grid_xgb.predict(X_test)

y_pred_orig = scaler_y.inverse_transform(y_pred)
y_test_orig = scaler_y.inverse_transform(y_test)

In [ ]:
k = X_test.shape[1]
n = len(X_test)

RMSE = float(format(np.sqrt(mean_squared_error(y_test_orig, y_pred_orig)),'.3f'))
MSE = mean_squared_error(y_test_orig, y_pred_orig)
MAE = mean_absolute_error(y_test_orig, y_pred_orig)
r2 = r2_score(y_test_orig, y_pred_orig)
adj_r2 = 1 - (1-r2) * (n-1)/(n-k)

print('RMSE =',RMSE, 
      '\nMSE =',MSE, 
      '\nMAE =',MAE,
      '\nR2=',r2, 
      '\nAdjusted R2 =',adj_r2)

## 11.Summary
Compare the MSE between Linear Regression, Random Forest, Artificial Neural Network and XGBoost tuned by Grid Search, the best model is XGBoost with the lowest MSE and highest R-squared value of 0.881.